In [248]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import openrouteservice as ors
import requests

In [249]:
API_KEY = '5b3ce3597851110001cf62484364c6151eb540e3949acae6888d47ef'

client = ors.Client(key=API_KEY)

## Visualizing bus routes

In [250]:
bus_stops = pd.read_csv('bus_stop_coords.csv', header=None)
bus_stops.rename(columns={0:"Bus Stop", 1:"x", 2:"y"}, inplace=True)
bus_stops

,Bus Stop,x,y
0,UTown,1.303555,103.774414
1,Museum,1.301081,103.773694
2,UHC,1.298913,103.776112
3,Opp UHC,1.298789,103.775627
4,Opp YIH,1.298973,103.774179
5,YIH,1.298898,103.774388
6,IT,1.297234,103.772697
7,CLB,1.296365,103.772261
8,Ventus,1.295335,103.770531
9,LT13,1.294727,103.770627


### All bus stops

In [251]:
map = folium.Map(location = [1.3083003040174188, 103.79569430095988], zoom_start = 15)
for i in range(len(bus_stops)):
    folium.Marker(location=[bus_stops.iloc[i, 1], bus_stops.iloc[i, 2]], colors='grey').add_to(map)
map

### A1 route

#### Operating hours
| Term | First Bus | Last Bus |
| --- | --- | --- |
| Weekday | 0715 | 2300 |
| Saturday | 0715 | 2300 |
| Sun/PH | 0900 | 2300 |

| Vacation | First Bus | Last Bus |
| --- | --- | --- |
| Weekday | 0715 | 2300 |
| Saturday | 0715 | 2300 |
| Sun/PH | 0900 | 2300 |

#### Frequency
| Weekday |  | Saturday |  | Sun/PH |  |
| --- | --- | --- | --- | --- | --- |
| 0715 - 0800 | 8-10 min | 0715 - 1930 | 15 min | 0900 - 2300 | 30 min |
| 0801 - 1000 | 4-6 min | 1931 - 2300 | 30 min | | |
| 1001 - 1100 | 10-12 min | | | | |
| 1101 - 1400 | 5-7 min | | | | |
| 1401 - 1715 | 8-10 min | | | | |
| 1716 - 1930 | 5-7 min | | | | |
| 1931 - 2300 | 15 min | | | | |

In [252]:
A1_bus = ['KR Bus Terminal', 'LT13', 'AS5', 'BIZ2', 'Opp TCOMS', 'PGP Terminal', 'KR MRT', 'LT27', 
          'University Hall', 'Opp UHC', 'YIH', 'CLB', 'KR Bus Terminal']

A1_stops = bus_stops[bus_stops['Bus Stop'].isin(A1_bus)]
A1_route_df = pd.DataFrame({'Bus Stop': A1_bus})
A1_route_df['route_index'] = A1_route_df.index

A1 = A1_route_df.merge(A1_stops, on='Bus Stop', how='left')
A1.loc[A1['route_index']==0, 'color'] = 'red'
A1.loc[A1['route_index']!=0, 'color'] = 'blue'
A1.loc[A1['route_index']==A1.shape[0]-1, 'color'] = 'red'
A1.drop(columns='route_index', inplace=True)


map = folium.Map(location = [1.2993920113270834, 103.77841451795645], zoom_start = 16)
for i in range(len(A1)):
    folium.Marker(location=[A1.iloc[i, 1], A1.iloc[i, 2]], icon=folium.Icon(color=A1.iloc[i, 3]), popup=A1.iloc[i, 0]).add_to(map)

dir_coord = [(A1.iloc[i,2], A1.iloc[i,1]) for i in range(len(A1))]
route = client.directions(coordinates=dir_coord, profile='driving-car', format='geojson', optimize_waypoints=True)
route_coords = [list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']]
folium.PolyLine(locations=route_coords,
                color='blue',
                weight=4,
                smooth_factor=0.1).add_to(map)

map

### A2 route

#### Operating hours
| Term | First Bus | Last Bus |
| --- | --- | --- |
| Weekday | 0715 | 2300 |
| Saturday | 0715 | 2300 |
| Sun/PH | 0900 | 2300 |

| Vacation | First Bus | Last Bus |
| --- | --- | --- |
| Weekday | 0715 | 2300 |
| Saturday | 0715 | 2300 |
| Sun/PH | 0900 | 2300 |

#### Frequency
| Weekday |  | Saturday |  | Sun/PH |  |
| --- | --- | --- | --- | --- | --- |
| 0715 - 1000 | 6-8 min | 0715 - 1930 | 15 min | 0900 - 2300 | 30 min |
| 1001 - 1115 | 9-11 min | 1931 - 2300 | 30 min | | |
| 1116 - 1400 | 6-8 min | | | | |
| 1401 - 1700 | 7-10 min | | | | |
| 1701 - 1930 | 6-8 min | | | | |
| 1931 - 2130 | 11-13 min | | | | |
| 2131 - 2300 | 15 min | | | | |

In [253]:
A2_bus = ['KR Bus Terminal', 'IT', 'Opp YIH', 'Museum', 'UHC', 'Opp University Hall', 'S17', 'Opp KR MRT', 
          'PGP Foyer', 'TCOMS', 'Opp HSSML', 'Opp NUSS', 'Ventus', 'KR Bus Terminal']

A2_stops = bus_stops[bus_stops['Bus Stop'].isin(A2_bus)]
A2_route_df = pd.DataFrame({'Bus Stop': A2_bus})
A2_route_df['route_index'] = A2_route_df.index

A2 = A2_route_df.merge(A2_stops, on='Bus Stop', how='left')
A2.loc[A2['route_index']==0, 'color'] = 'red'
A2.loc[A2['route_index']!=0, 'color'] = 'blue'
A2.loc[A2['route_index']==A2.shape[0]-1, 'color'] = 'red'
A2.drop(columns='route_index', inplace=True)


map = folium.Map(location = [1.2993920113270834, 103.77841451795645], zoom_start = 16)
for i in range(len(A2)):
    folium.Marker(location=[A2.iloc[i, 1], A2.iloc[i, 2]], icon=folium.Icon(color=A2.iloc[i, 3]), popup=A2.iloc[i, 0]).add_to(map)

dir_coord = [(A2.iloc[i,2], A2.iloc[i,1]) for i in range(len(A2))]
route = client.directions(coordinates=dir_coord, profile='driving-car', format='geojson', optimize_waypoints=True)
route_coords = [list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']]
folium.PolyLine(locations=route_coords,
                color='blue',
                weight=4,
                smooth_factor=0.1).add_to(map)

map

### Getting accurate route data from Mapbox API

In [254]:
museum_to_utown_url = 'https://api.mapbox.com/directions/v5/mapbox/driving/103.773474%2C1.301166%3B103.774422%2C1.303533?alternatives=false&geometries=geojson&language=en&overview=full&steps=true&access_token=pk.eyJ1IjoiZ3VvaG9uZ3lpMTExIiwiYSI6ImNtMng0OHc4cTAwenMybG9iczg4cjBoNjcifQ.tc6o1kU_mTemKQhbVy5mNA'
response = requests.get(museum_to_utown_url)
museum_to_utown = response.json()

utown_to_yih_url = 'https://api.mapbox.com/directions/v5/mapbox/driving/103.774422%2C1.303533%3B103.774308%2C1.298869?alternatives=false&geometries=geojson&language=en&overview=full&steps=true&access_token=pk.eyJ1IjoiZ3VvaG9uZ3lpMTExIiwiYSI6ImNtMng0OHc4cTAwenMybG9iczg4cjBoNjcifQ.tc6o1kU_mTemKQhbVy5mNA'
yih_response = requests.get(utown_to_yih_url)
utown_to_yih = yih_response.json()

utown_to_uhc_url = 'https://api.mapbox.com/directions/v5/mapbox/driving/103.774422%2C1.303533%3B103.776055%2C1.298898?alternatives=false&geometries=geojson&language=en&overview=full&steps=true&access_token=pk.eyJ1IjoiZ3VvaG9uZ3lpMTExIiwiYSI6ImNtMng0OHc4cTAwenMybG9iczg4cjBoNjcifQ.tc6o1kU_mTemKQhbVy5mNA'
uhc_response = requests.get(utown_to_uhc_url)
utown_to_uhc = uhc_response.json()

### D1 route

#### Operating hours
| Term | First Bus | Last Bus |
| --- | --- | --- |
| Weekday | 0715 | 2300 |
| Saturday | 0720 | 2300 |
| Sun/PH | 0900 | 2300 |

| Vacation | First Bus | Last Bus |
| --- | --- | --- |
| Weekday | 0715 | 2300 |
| Saturday | 0720 | 2300 |
| Sun/PH | 0900 | 2300 |

#### Frequency
| Weekday |  | Saturday |  | Sun/PH |  |
| --- | --- | --- | --- | --- | --- |
| 0715 - 0800 | 10-12 min | 0720 - 1930 | 15 min | 0900 - 2300 | 30 min |
| 0801 - 1930 | 6-8 min | 1931 - 2300 | 30 min | | |
| 1931 - 2130 | 11-13 min | | | | |
| 2131 - 2300 | 15 min | | | | |

In [255]:
D1_bus = ['COM3', 'Opp HSSML', 'Opp NUSS', 'Ventus', 'IT', 'Opp YIH', 'Museum', 'UTown', 'YIH', 'CLB', 'LT13', 'AS5', 'BIZ2', 'COM3']

D1_stops = bus_stops[bus_stops['Bus Stop'].isin(D1_bus)]
D1_route_df = pd.DataFrame({'Bus Stop': D1_bus})
D1_route_df['route_index'] = D1_route_df.index

D1 = D1_route_df.merge(D1_stops, on='Bus Stop', how='left')
D1.loc[D1['route_index']==0, 'color'] = 'red'
D1.loc[D1['route_index']!=0, 'color'] = 'blue'
D1.loc[D1['route_index']==D1.shape[0]-1, 'color'] = 'red'
D1.drop(columns='route_index', inplace=True)


map = folium.Map(location = [1.2993920113270834, 103.77841451795645], zoom_start = 16)
for i in range(len(D1)):
    folium.Marker(location=[D1.iloc[i, 1], D1.iloc[i, 2]], icon=folium.Icon(color=D1.iloc[i, 3]), popup=D1.iloc[i, 0]).add_to(map)

# COM3 to Museum
dir_coord_museum = [(D1.iloc[i,2], D1.iloc[i,1]) for i in range(7)]
route = client.directions(coordinates=dir_coord_museum, profile='driving-car', format='geojson')
route_coords_museum = [list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']]
folium.PolyLine(locations=route_coords_museum,
                color='blue',
                weight=4,
                smooth_factor=0.1).add_to(map)

# Museum to UTown
route_coords = [list(reversed(coord)) for coord in museum_to_utown['routes'][0]['geometry']['coordinates']]
folium.PolyLine(locations=route_coords,
                color='blue',
                weight=4,
                smooth_factor=0.1).add_to(map)

# UTown to YIH
route_coords = [list(reversed(coord)) for coord in utown_to_yih['routes'][0]['geometry']['coordinates']]
folium.PolyLine(locations=route_coords,
                color='blue',
                weight=4,
                smooth_factor=0.1).add_to(map)

# YIH to COM3
dir_coord_rem = [(D1.iloc[i,2], D1.iloc[i,1]) for i in range(8, len(D1))]
route = client.directions(coordinates=dir_coord_rem, profile='driving-car', format='geojson')
route_coords_rem = [list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']]
folium.PolyLine(locations=route_coords_rem,
                color='blue',
                weight=4,
                smooth_factor=0.1).add_to(map)


map

### D2 route

#### Operating hours
| Term | First Bus | Last Bus |
| --- | --- | --- |
| Weekday | 0715 | 2300 |
| Saturday | 0720 | 2300 |
| Sun/PH | 0905 | 2300 |

| Vacation | First Bus | Last Bus |
| --- | --- | --- |
| Weekday | 0715 | 2300 |
| Saturday | 0720 | 2300 |
| Sun/PH | 0905 | 2300 |

#### Frequency
| Weekday |  | Saturday |  | Sun/PH |  |
| --- | --- | --- | --- | --- | --- |
| 0715 - 1000 | 5-7 min | 0720 - 1930 | 15 min | 0900 - 2300 | 30 min |
| 1001 - 1115 | 9-11 min | 1931 - 2300 | 30 min | | |
| 1116 - 1400 | 5-7 min | | | | |
| 1401 - 1715 | 7-9 min | | | | |
| 1716 - 1930 | 5-7 min | | | | |
| 1931 - 2130 | 10-12 min | | | | |
| 2131 - 2300 | 15 min | | | | |

In [256]:
D2_bus = ['COM3', 'Opp TCOMS', 'PGP Terminal', 'KR MRT', 'LT27', 'University Hall', 'Opp UHC', 'Museum', 
          'UTown', 'UHC', 'Opp University Hall', 'S17', 'Opp KR MRT', 'PGP Foyer', 'TCOMS', 'COM3']

D2_stops = bus_stops[bus_stops['Bus Stop'].isin(D2_bus)]
D2_route_df = pd.DataFrame({'Bus Stop': D2_bus})
D2_route_df['route_index'] = D2_route_df.index

D2 = D2_route_df.merge(D2_stops, on='Bus Stop', how='left')
D2.loc[D2['route_index']==0, 'color'] = 'red'
D2.loc[D2['route_index']!=0, 'color'] = 'blue'
D2.loc[D2['route_index']==D2.shape[0]-1, 'color'] = 'red'
D2.drop(columns='route_index', inplace=True)


map = folium.Map(location = [1.2993920113270834, 103.77841451795645], zoom_start = 16)
for i in range(len(D2)):
    folium.Marker(location=[D2.iloc[i, 1], D2.iloc[i, 2]], icon=folium.Icon(color=D2.iloc[i, 3]), popup=D2.iloc[i, 0]).add_to(map)

# # split routes into 3 segments so that the entire route is accurate

# COM3 to Museum
dir_coord_museum = [(D2.iloc[i,2], D2.iloc[i,1]) for i in range(8)]
route = client.directions(coordinates=dir_coord_museum, profile='driving-car', format='geojson')
route_coords_museum = [list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']]

folium.PolyLine(locations=route_coords_museum,
                color='blue',
                weight=4,
                smooth_factor=0.1).add_to(map)

# Museum to UTown
route_coords = [list(reversed(coord)) for coord in museum_to_utown['routes'][0]['geometry']['coordinates']]
folium.PolyLine(locations=route_coords,
                color='blue',
                weight=4,
                smooth_factor=0.1).add_to(map)

# UTown to UHC
route_coords = [list(reversed(coord)) for coord in utown_to_uhc['routes'][0]['geometry']['coordinates']]
folium.PolyLine(locations=route_coords,
                color='blue',
                weight=4,
                smooth_factor=0.1).add_to(map)

# UHC to COM3
dir_coord_rem = [(D2.iloc[i,2], D2.iloc[i,1]) for i in range(9, len(D2))]
route = client.directions(coordinates=dir_coord_rem, profile='driving-car', format='geojson')
route_coords_rem = [list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']]
folium.PolyLine(locations=route_coords_rem,
                color='blue',
                weight=4,
                smooth_factor=0.1).add_to(map)

map


### BTC route

#### Operating hours
| Term | First Bus | Last Bus |
| --- | --- | --- |
| Weekday | 0730 | 2140 |
| Saturday | No service |
| Sun/PH | No service |

| Vacation | First Bus | Last Bus |
| --- | --- | --- |
| Weekday | 0730 | 1940 |
| Saturday | No service |
| Sun/PH | No service |

#### Frequency
| Weekday |  | Saturday |  Sun/PH |  
| --- | --- | --- | --- |  
| 0730 - 1000 | 30 min | No service |  No service |
| 1001 - 1110 | 35 min | 
| 1111 - 1410 | 30 min | 
| 1401 - 1710 | 45 min | 
| 1711 - 1940 | 30 min | 
| 1941 - 2110 | 45 min | 
| 2111 - 2140 | 30 min | 

In [257]:
btc_url = 'https://api.mapbox.com/directions/v5/mapbox/driving/103.817832%2C1.319848%3B103.818893%2C1.320928%3B103.815903%2C1.32268%3B103.784429%2C1.294841%3B103.780988%2C1.297413%3B103.778766%2C1.297204%3B103.775656%2C1.29881%3B103.774434%2C1.303535%3B103.772749%2C1.301005%3B103.769636%2C1.301896%3B103.769058%2C1.302245%3B103.773469%2C1.301136%3B103.774308%2C1.298863%3B103.772194%2C1.296321%3B103.770581%2C1.294841%3B103.771738%2C1.293506%3B103.775121%2C1.29325%3B103.780403%2C1.291822%3B103.816278%2C1.323278%3B103.81827%2C1.319337?alternatives=false&geometries=geojson&language=en&overview=full&steps=true&access_token=pk.eyJ1IjoiZ3VvaG9uZ3lpMTExIiwiYSI6ImNtMng0OHc4cTAwenMybG9iczg4cjBoNjcifQ.tc6o1kU_mTemKQhbVy5mNA'
response = requests.get(btc_url)
btc_route = response.json()

In [258]:
BTC_bus = ['Oei Tiong Ham Building (BTC)', 'Botanic Gardens MRT (BTC)', 'KR MRT', 'LT27', 'University Hall', 'Opp UHC', 
           'UTown', 'Raffles Hall', 'Kent Vale', 'Museum', 'YIH', 'CLB', 'LT13', 'AS5', 'BIZ2', 'PGP Terminal', 'College Green (BTC)', 
           'Oei Tiong Ham Building (BTC)']

BTC_stops = bus_stops[bus_stops['Bus Stop'].isin(BTC_bus)]
BTC_route_df = pd.DataFrame({'Bus Stop': BTC_bus})
BTC_route_df['route_index'] = BTC_route_df.index

BTC = BTC_route_df.merge(BTC_stops, on='Bus Stop', how='left')
BTC.loc[BTC['route_index']==0, 'color'] = 'red'
BTC.loc[BTC['route_index']!=0, 'color'] = 'blue'
BTC.loc[BTC['route_index']==BTC.shape[0]-1, 'color'] = 'red'
BTC.drop(columns='route_index', inplace=True)

map = folium.Map(location = [1.2993920113270834, 103.77841451795645], zoom_start = 14)
for i in range(len(BTC)):
    folium.Marker(location=[BTC.iloc[i, 1], BTC.iloc[i, 2]], icon=folium.Icon(color=BTC.iloc[i, 3]), popup=BTC.iloc[i, 0]).add_to(map)

route_coords = [list(reversed(coord)) for coord in btc_route['routes'][0]['geometry']['coordinates']]
folium.PolyLine(locations=route_coords,
                color='blue',
                weight=4,
                smooth_factor=0.1).add_to(map)

map

### E route

#### Operating hours
| Term | First Bus | Last Bus |
| --- | --- | --- |
| Weekday | 0800 | 1600 |
| Saturday | No service |
| Sun/PH | No service |

#### Frequency
| Weekday |  | Saturday | Sun/PH |
| --- | --- | --- | --- |
| 0800 - 1000 | 15 min | No service | No service |
| 1115 - 1400 | 15 min |
| 1515 - 1600 | 15 min |

In [261]:
E_url = 'https://api.mapbox.com/directions/v5/mapbox/driving/103.774437%2C1.303529%3B103.772744%2C1.301%3B103.769629%2C1.301897%3B103.769053%2C1.302199%3B103.770144%2C1.300623%3B103.769865%2C1.297757%3B103.77271%2C1.297189%3B103.774132%2C1.298847%3B103.774415%2C1.303511?alternatives=false&geometries=geojson&language=en&overview=full&steps=true&access_token=pk.eyJ1IjoiZ3VvaG9uZ3lpMTExIiwiYSI6ImNtMng0OHc4cTAwenMybG9iczg4cjBoNjcifQ.tc6o1kU_mTemKQhbVy5mNA'
responses = requests.get(E_url)
E_route = responses.json()

In [262]:
E_bus = ['UTown', 'Raffles Hall', 'Kent Vale', 'EA', 'SDE3', 'IT', 'Opp YIH', 'UTown']

E_stops = bus_stops[bus_stops['Bus Stop'].isin(E_bus)]
E_route_df = pd.DataFrame({'Bus Stop': E_bus})
E_route_df['route_index'] = E_route_df.index

E = E_route_df.merge(E_stops, on='Bus Stop', how='left')
E.loc[E['route_index']==0, 'color'] = 'red'
E.loc[E['route_index']!=0, 'color'] = 'blue'
E.loc[E['route_index']==E.shape[0]-1, 'color'] = 'red'
E.drop(columns='route_index', inplace=True)


map = folium.Map(location = [1.3002841404515253, 103.77231795009253], zoom_start = 17)
for i in range(len(E)):
    folium.Marker(location=[E.iloc[i, 1], E.iloc[i, 2]], icon=folium.Icon(color=E.iloc[i, 3]), popup=E.iloc[i, 0]).add_to(map)


route_coords = [list(reversed(coord)) for coord in E_route['routes'][0]['geometry']['coordinates']]
folium.PolyLine(locations=route_coords,
                color='blue',
                weight=4,
                smooth_factor=0.1).add_to(map)

map

### K route

#### Operating hours
| Term/Vacation | First Bus | Last Bus |
| --- | --- | --- |
| Weekday | 0700 | 2300 |
| Saturday | 0700 | 1930 |
| Sun/PH | No service |

#### Frequency
| Weekday |  | Saturday | | Sun/PH |
| --- | --- | --- | --- | --- |
| 0700 - 2300 | 40 min | 0700 - 1930 | 40 min | No service |

In [263]:
K_url = 'https://api.mapbox.com/directions/v5/mapbox/driving/103.78039%2C1.2918%3B103.784424%2C1.294834%3B103.780996%2C1.297405%3B103.778771%2C1.297191%3B103.775658%2C1.298817%3B103.774306%2C1.298857%3B103.772199%2C1.296316%3B103.769637%2C1.297888%3B103.769957%2C1.300739%3B103.769629%2C1.301901%3B103.769015%2C1.302229%3B103.773468%2C1.301142%3B103.77606%2C1.298901%3B103.778076%2C1.297504%3B103.780704%2C1.297515%3B103.784574%2C1.294956%3B103.781073%2C1.290993?alternatives=false&geometries=geojson&language=en&overview=full&steps=true&access_token=pk.eyJ1IjoiZ3VvaG9uZ3lpMTExIiwiYSI6ImNtMng0OHc4cTAwenMybG9iczg4cjBoNjcifQ.tc6o1kU_mTemKQhbVy5mNA'
responses = requests.get(K_url)
K_route = responses.json()

In [264]:
K_bus = ['PGP Terminal', 'KR MRT', 'LT27', 'University Hall', 'Opp UHC', 'YIH', 'CLB', 'Opp SDE3', 'The Japanese Primary School', 
         'Kent Vale', 'Museum', 'UHC', 'Opp University Hall', 'S17', 'Opp KR MRT', 'PGP Foyer']

K_stops = bus_stops[bus_stops['Bus Stop'].isin(K_bus)]
K_route_df = pd.DataFrame({'Bus Stop': K_bus})
K_route_df['route_index'] = K_route_df.index

K = K_route_df.merge(K_stops, on='Bus Stop', how='left')
K.loc[K['route_index']==0, 'color'] = 'red'
K.loc[K['route_index']!=0, 'color'] = 'blue'
K.loc[K['route_index']==K.shape[0]-1, 'color'] = 'red'
K.drop(columns='route_index', inplace=True)

map = folium.Map(location = [1.2993920113270834, 103.77841451795645], zoom_start = 16)
for i in range(len(K)):
    folium.Marker(location=[K.iloc[i, 1], K.iloc[i, 2]], icon=folium.Icon(color=K.iloc[i, 3]), popup=K.iloc[i, 0]).add_to(map)

route_coords = [list(reversed(coord)) for coord in K_route['routes'][0]['geometry']['coordinates']]
folium.PolyLine(locations=route_coords,
                color='blue',
                weight=4,
                smooth_factor=0.1).add_to(map)

map